## WeatherPy 
### Generate 500+ random latitudes and longitudes and use Weathermap.org and API keys to retrieve and graph weather day showing various relationships. 

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from citipy import citipy
from random import uniform
from config import api_key

In [2]:
# Create a df to lat, lon 
column_names = ["City", "Country"]
cities_df = pd.DataFrame(columns=column_names)
cities_df

,City,Country


## Generate Cites List from Lats and Longs

In [3]:
# Randomly generate a list of 1500 cities. Use 50 to test for duplicates.
for x in range(250):
    lat, lng = np.random.uniform(low=-90.000, high=90.000), np.random.uniform(low=-180.000, high=180.000)
    city = citipy.nearest_city(lat, lng)
    cities_df = cities_df.append({"City": city.city_name,"Country": city.country_code,}, ignore_index=True)
cities_df.head()

,City,Country
0,mataura,pf
1,albany,au
2,opuwo,na
3,beloha,mg
4,airai,pw


In [4]:
#Remove duplicate cities
cities_df = cities_df.drop_duplicates(subset='City').reset_index()
cities_df.count()

index      150
City       150
Country    150
dtype: int64

## Perform API Calls

In [5]:
#Create a df to add all the data from the API calls.
column_names = ["City", "Country","Temperature (F)",
                "Latitude","Longitude","Humidity (%)",
                "Cloudiness (%)", "Wind Speed (mph)"]
clean_cities_df = pd.DataFrame(columns=column_names)
clean_cities_df.head()

,City,Country,Temperature (F),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)


In [6]:
# Loop through the list of cities and add the data to the df.
url = "http://api.openweathermap.org/data/2.5/weather?"

row_count = 1

print("Beginning Data Retrieval")
for index, row in cities_df.iterrows():
    target_city = row["City"]
    target_url = url + "appid=" + api_key + "&units=IMPERIAL" + "&q=" + target_city.replace(" ","")
    city_data = requests.get(target_url).json()
    if city_data["cod"] == "404":
        print("City not found, skipping...")

    else:
        clean_cities_df.set_value(index, "City", city_data["name"])
        clean_cities_df.set_value(index, "Country", city_data["sys"]["country"])
        clean_cities_df.set_value(index, "Temperature (F)", city_data["main"]["temp"])
        clean_cities_df.set_value(index, "Latitude", city_data["coord"]["lat"])
        clean_cities_df.set_value(index, "Longitude", city_data["coord"]["lon"])
        clean_cities_df.set_value(index, "Humidity (%)", city_data["main"]["humidity"])
        clean_cities_df.set_value(index, "Cloudiness (%)", city_data["clouds"]["all"])
        clean_cities_df.set_value(index, "Wind Speed (mph)", city_data["wind"]["speed"])
        
        print("------------------------")
        print("Proceesing: City # " , row_count, ' | ' , city_data["name"], city_data["sys"]["country"])
        print(target_url)
        row_count += 1

Beginning Data Retrieval
------------------------
Proceesing: City #  1  |  Mathura IN
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=mataura
------------------------
Proceesing: City #  2  |  Albany US
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=albany
------------------------
Proceesing: City #  3  |  Opuwo NA
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=opuwo
------------------------
Proceesing: City #  4  |  Beloha MG
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=beloha
------------------------
Proceesing: City #  5  |  Aura FI
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=airai
------------------------
Proceesing: City #  6  |  Liwale TZ
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9

------------------------
Proceesing: City #  46  |  Dikson RU
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=dikson
------------------------
Proceesing: City #  47  |  Daugavpils LV
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=daugavpils
------------------------
Proceesing: City #  48  |  Zonguldak TR
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=zonguldak
------------------------
Proceesing: City #  49  |  Chuy UY
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=chuy
City not found, skipping...
City not found, skipping...
------------------------
Proceesing: City #  50  |  Healdsburg US
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=healdsburg
------------------------
Proceesing: City #  51  |  Vaini TO
http:

------------------------
Proceesing: City #  91  |  Avarua CK
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=avarua
------------------------
Proceesing: City #  92  |  Sitka US
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=sitka
------------------------
Proceesing: City #  93  |  Mahebourg MU
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=mahebourg
City not found, skipping...
------------------------
Proceesing: City #  94  |  Acarau BR
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=acarau
------------------------
Proceesing: City #  95  |  Hukuntsi BW
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=hukuntsi
------------------------
Proceesing: City #  96  |  Atherton GB
http://api.openweathermap.org/data/2.5/w

In [7]:
clean_cities_df

,City,Country,Temperature (F),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Mathura,IN,67.16,27.5,77.68,61,0,10.67
1,Albany,US,34.36,42.65,-73.76,51,40,19.46
2,Opuwo,NA,89.89,-18.06,13.84,16,0,3.74
3,Beloha,MG,71.66,-25.17,45.05,83,88,14.92
4,Aura,FI,39.2,60.62,22.58,82,0,11.41
5,Liwale,TZ,91.78,-9.77,37.93,30,0,6.76
6,Busselton,AU,63.02,-33.65,115.33,100,12,2.28
7,Jawa,JO,69.8,31.85,35.94,22,0,11.41
8,Butaritari,KI,82.55,3.07,172.79,100,68,7.31
9,Hilo,US,50.2,19.73,-155.09,77,1,8.05


In [8]:
#del cities_df['index']
clean_cities_df = clean_cities_df.reset_index()
del clean_cities_df['index']
clean_cities_df.head()

,City,Country,Temperature (F),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Mathura,IN,67.16,27.5,77.68,61,0,10.67
1,Albany,US,34.36,42.65,-73.76,51,40,19.46
2,Opuwo,NA,89.89,-18.06,13.84,16,0,3.74
3,Beloha,MG,71.66,-25.17,45.05,83,88,14.92
4,Aura,FI,39.2,60.62,22.58,82,0,11.41


In [9]:
# Save data to a CSV file
clean_cities_df.to_csv("City_Data.csv")